In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
df_mock = load_data(path)
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_rsv = load_data(path)
df_mock.shape, df_rsv.shape

In [ ]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/UTMB6.xlsx'
df_intact = load_data(path)
df_intact.shape

In [ ]:
df_mock_sample = df_mock.copy()
df_rsv_sample = df_rsv.copy()
df_mock_sample.shape, df_rsv_sample.shape

In [ ]:
df_ngs_src = pd.read_excel('~/Downloads/EJ_rsv.xlsx')
df_ngs_src.shape, df_ngs_src.head()

In [ ]:
mass_glu = 147.1

def reg_mass(seq):
    """given a sequence(contains ACGU only), calculate the intact mass
    """
    mass = 0
    bases = {'A': 329.0525,
            'C': 305.0413,
            'G': 345.0474,
            'U': 306.0253}
    
    for base in bases:
        num = seq.count(base)
        mass += num * bases.get(base)

    mass += H2O
    return mass


In [ ]:
dft = df_ngs_src.dropna(subset=['seq']).copy()
df_sample = df_rsv_sample
for idx, row in dft.iterrows():
    seq = row['seq']
#     print(idx, row['seqs'])
    mass = reg_mass(seq)
    df_ngs_src.loc[idx, 'MassReg'] = mass + C+C+A if len(seq)>=72 else 0
    df_ngs_src.loc[idx, 'MassP'] = mass + 80
    df_ngs_src.loc[idx, 'MassOH'] = mass - 80
    df_ngs_src.loc[idx, 'MassAmino'] = mass + mass_glu - H2O

In [ ]:
df_ngs_src.head()

In [ ]:
df = df_ngs_src.copy()
df.rename(columns={'MassP': "Mass"}, inplace=True)

In [ ]:
df[['name', 'b', 'seqs', 'Vol', 'Mass']]

In [ ]:
dfm_seq, dfm = peer_dfs(df, df_intact)
dfm_seq.shape, dfm.shape

In [ ]:
plotly_zones(df, dfm_seq, x='Mass', y='Vol')
plotly_zones(df_intact, dfm)

In [ ]:

print(df_intact[(df_intact.Mass>9690)&(df_intact.Mass<14550)].sort_values('Mass'))
print(dfm.sort_values('Mass'))

In [ ]:
bcr = base_calling_random(dfm)
plotly_basecalling(dfm, bcr[1])

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

def shift_hits(df_ngs, df_src, shifts):
    PROCESSES = 8
    params = [(df_ngs, df_src, shift) for shift in shifts]
    with multiprocessing.Pool(PROCESSES) as pool:
        df_basecallings = pool.starmap(func, params)

    df_hits = pd.concat(df_basecallings).drop_duplicates()
    return df_hits


In [ ]:
dft = dfm_seq.copy()
df_sample = df_rsv
shifts = [0, 14]
for idx, row in dft.iterrows():
    seq = row['seq']
    df_5p = pd.read_json(row['MassLadder5p'])
    df_5p['Base'] = list(seq)
    df_5p['Idx'] = np.array(df_5p.index)+1
    
    df_hits_5p = shift_hits(df_5p, df_sample, shifts)
    dfm_seq.loc[idx, 'Hit5Count'] = df_hits_5p.shape[0]
    dfm_seq.loc[idx, 'Hit5Ladder'] = df_hits_5p.to_json()

dfm_seq.to_excel('~/Downloads/EJ_rsv_result.xlsx')

In [ ]:
dft = pd.read_json(dfm_seq.loc[17, 'MassLadder5p'])
df_hit = shift_hits(dft, df_rsv, [0, 14, 16, 18])
plotly_zone(df_hit, y='Shift')
dfm_seq.loc[17]